In [3]:
import duckdb
import pandas as pd

con = duckdb.connect("dev_database.duckdb")
con.sql("USE dev_database.dev_sode")
con.sql("SHOW TABLES;").show()


┌───────────────────────────────────────────┐
│                   name                    │
│                  varchar                  │
├───────────────────────────────────────────┤
│ fact_daily_climate_parameters             │
│ ghcnd_2000                                │
│ ghcnd_2001                                │
│ ghcnd_2002                                │
│ ghcnd_2003                                │
│ ghcnd_2004                                │
│ ghcnd_2005                                │
│ ghcnd_2006                                │
│ ghcnd_2007                                │
│ ghcnd_2008                                │
│     ·                                     │
│     ·                                     │
│     ·                                     │
│ ghcnd_2018                                │
│ ghcnd_2019                                │
│ ghcnd_2020                                │
│ ghcnd_2021                                │
│ ghcnd_2022                      

In [4]:
df = con.sql("select * from fact_daily_climate_parameters ").df()

In [5]:
df.head()

,reading_date,source_flag,measurement_flag,precipitation_mm,daily_max_temperature_celsius,daily_min_temperature_celsius,snow_fall_mm,snow_depth_mm
0,2024-05-21,U,H,NaN,16.7,1.1,NaN,NaN
1,2024-05-20,S,D,0.0,NaN,NaN,NaN,NaN
2,2024-05-20,U,H,NaN,20.6,6.1,NaN,NaN
3,2024-05-20,S,B,0.0,NaN,NaN,NaN,NaN
4,2024-05-19,U,H,NaN,21.7,5.6,NaN,NaN


## Stations

In [13]:
import boto3
import pandas as pd
import re
bucket_name = 'noaa-ghcn-pds'
key = 'ghcnd-stations.txt'

client = boto3.client('s3')

text_obj = client.get_object(Bucket=bucket_name, Key=key)
text_body = text_obj['Body'].read().decode('utf-8')
#text_csv = re.sub(r'\s+',',',text_body)

# text_body.split('\n')
with open('stations.txt','w') as f:
    f.write(text_body)




NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [7]:
df_stations = pd.read_fwf('stations.txt')
df_stations.columns=['station_id','latitude','longitude','state','station_name','gsn_flag','hcn_crn_flag','wmo_number']
df_stations.to_csv('stations.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 3964: invalid continuation byte

## Countries

In [8]:
import boto3
import pandas as pd
import re
bucket_name = 'noaa-ghcn-pds'
key = 'ghcnd-countries.txt'

client = boto3.client('s3')

text_obj = client.get_object(Bucket=bucket_name, Key=key)
text_body = text_obj['Body'].read().decode('utf-8')
#text_csv = re.sub(r'\s+',',',text_body)

# text_body.split('\n')
with open('countries.txt','w') as f:
    f.write(text_body)




In [9]:
df_countries = pd.read_fwf('countries.txt')
df_countries.columns=['country_code','country_name']
df_countries.to_csv('countries.csv')

# States

In [10]:
import boto3
import pandas as pd
import re
bucket_name = 'noaa-ghcn-pds'
key = 'ghcnd-states.txt'

client = boto3.client('s3')

text_obj = client.get_object(Bucket=bucket_name, Key=key)
text_body = text_obj['Body'].read().decode('utf-8')
#text_csv = re.sub(r'\s+',',',text_body)

# text_body.split('\n')
with open('state.txt','w') as f:
    f.write(text_body)

df_states = pd.read_fwf('state.txt')
df_states.columns=['state_code','state_name']
df_states.to_csv('states.csv')

In [11]:
# df_stations= df.join(df_countries,on=)
def push_metadata(data, database_name, table_name):

    conn = duckdb.connect(database=database_name,read_only=False)
    conn.execute("CREATE SCHEMA IF NOT EXISTS dev_sode;")
    df = pd.read_csv(data)

    conn.execute(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")

    conn.close()

In [12]:
push_metadata('states.csv', "dev_database.duckdb", "states")
push_metadata('stations.csv',"dev_database.duckdb", "stations")
push_metadata('countries.csv', "dev_database.duckdb", "countries")

FileNotFoundError: [Errno 2] No such file or directory: 'stations.csv'